In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import os
import pandas as pd

# ## Keras
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator


In [2]:
columns = ['Image Path', 'Label']
num_bins = 25
samples_per_bin = 200

In [3]:
def load_dataset(dirs):
    data = []
    for directory in dirs:
        csv_path = os.path.join(directory, 'captured_data.csv')
        df = pd.read_csv(csv_path)
        data.append(df)
    return pd.concat(data, ignore_index=True)

In [4]:
# Hyperparameters
input_shape = (224, 224, 3)  # Modify the input shape based on your chosen pre-trained model
num_classes = 1  # Number of object classes, change accordingly
root_directory='D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\ '

In [5]:
# Load the dataset
dirs = [
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230730_233758',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_225555',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_225500',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_223453',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_223329',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_223213',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_223107',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_222917',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_222800',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_222634',
    'D:\Master\ProjectOutsideOfCouseScope\ObjectDetection\captures_20230731_194859',
]

In [6]:
dataset = load_dataset(dirs)

print(dataset)



                                      Image Path        Label
0     captures_20230730_233758\WateringCan_0.png  WateringCan
1     captures_20230730_233758\WateringCan_1.png  WateringCan
2     captures_20230730_233758\WateringCan_2.png  WateringCan
3     captures_20230730_233758\WateringCan_3.png  WateringCan
4     captures_20230730_233758\WateringCan_4.png  WateringCan
...                                          ...          ...
4302     captures_20230731_194859\Bottle_417.png       Bottle
4303     captures_20230731_194859\Bottle_418.png       Bottle
4304     captures_20230731_194859\Bottle_419.png       Bottle
4305     captures_20230731_194859\Bottle_420.png       Bottle
4306     captures_20230731_194859\Bottle_421.png       Bottle

[4307 rows x 2 columns]


In [7]:
# Split dataset into train and test sets (you can also use validation set)
train_split = 0.8
train_size = int(train_split * len(dataset))
train_df = dataset[:train_size]
test_df = dataset[train_size:]

In [8]:
train_df

,Image Path,Label
0,captures_20230730_233758\WateringCan_0.png,WateringCan
1,captures_20230730_233758\WateringCan_1.png,WateringCan
2,captures_20230730_233758\WateringCan_2.png,WateringCan
3,captures_20230730_233758\WateringCan_3.png,WateringCan
4,captures_20230730_233758\WateringCan_4.png,WateringCan
...,...,...
3440,captures_20230731_222634\Mug_31.png,Mug
3441,captures_20230731_222634\Mug_32.png,Mug
3442,captures_20230731_222634\Mug_33.png,Mug
3443,captures_20230731_222634\Mug_34.png,Mug


In [9]:
# Define the image directory and image size
image_dir = 'D:/Master/ProjectOutsideOfCouseScope/ObjectDetection/'
image_size = (416, 416)  # YOLO typically uses 416x416 input size

In [11]:
# Create an ImageDataGenerator for data augmentation and loading images
data_generator = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
train_data = data_generator.flow_from_dataframe(
    dataframe=dataset,
    directory=image_dir,
    x_col='Image Path',
    y_col='Label',
    subset='training',
    batch_size=32,
    target_size=image_size,
    class_mode='categorical'
)
val_data = data_generator.flow_from_dataframe(
    dataframe=dataset,
    directory=image_dir,
    x_col='Image Path',
    y_col='Label',
    subset='validation',
    batch_size=32,
    target_size=image_size,
    class_mode='categorical'
)

Found 3446 validated image filenames belonging to 11 classes.
Found 861 validated image filenames belonging to 11 classes.


In [21]:
import cv2

In [25]:

# Load YOLO model
model = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")